# CAR ACCIDENT SEVERITY
## Introduction



Car accidents are one of the leading causes of loss of lives and destruction of property. In this study we seek to study the common causes of car accidents and how they can be prevented. This analysis seeks to enable the drivers to be more careful depending on the weather and road conditions on any given day or a way for the police to enforce more safety protocols. To attain this we shall use Collisions Data from City of Seattle’s police department showing all the collisions from 2004.

## Data

We shall use an extensive dataset from City of Seattle’s Police Department showing collisons from 2004 to date. To build a model to prevent future accidents and/or reduce their severity, we will use the following attributes — ADDRTYPE, WEATHER, ROADCOND, VEHCOUNT, PERSONCOUNT.